In [4]:
import pandas as pd
import numpy as np
import functools
#from functools import partial

import matplotlib.pyplot as plt
from itertools import count

from bokeh.plotting import figure, show, output_notebook
from bokeh.io import output_notebook

#output_fileはHtml用なのでJupytherNotebook上でデータを確認するときには不要,HTMLができてしまって帰って見にくい
#from bokeh.plotting import output_file,

In [69]:
df = pd.read_csv("Log_DM1_single_out.csv")

In [70]:
#hexToDec()でのSettingWithCopyWarning回避の為 Paramter = Noneを指定する
pd.set_option('mode.chained_assignment',None)

In [71]:
df.head(5)

,Time,Node,CANID,DLC,Byte1,Byte2,Byte3,Byte4,Byte5,Byte6,Byte7,Byte8
0,0.002803,1,CF00400x,8,00,80,7D,00,00,FF,FF,FF
1,0.003303,1,488,8,00,00,00,00,00,00,00,00
2,0.003799,1,512,8,00,00,00,00,00,00,00,00
3,0.004299,1,108,8,00,00,00,00,00,00,00,00
4,0.005067,1,208,8,FF,FE,32,00,00,00,00,32


In [8]:
df.dtypes

Time     float64
Node       int64
CANID     object
DLC        int64
Byte1     object
Byte2     object
Byte3     object
Byte4     object
Byte5     object
Byte6     object
Byte7     object
Byte8     object
dtype: object

In [72]:
"""-------------------------------------------
16進数から10進数へ変換するための関数
hexToDec(hexCAN,hexCANtrg)
引数 :　
        ①hexCAN : Hex値が入ったseries(列の配列)
        ②hexCANtrg : ターゲット用のseries
戻り値:
        hexCANtrg : ターゲット用のseries
-------------------------------------------"""

def hexToDec(hexCAN,hexCANtrg):
    for ix, elem in enumerate(hexCAN):
        if isinshexToDectance(elem,int):
            hexCANtrg[ix] = hexCAN[ix] 
        
        else :
            hexCANtrg[ix] = int(hexCAN[ix],16)
    return hexCANtrg

In [24]:
#df['Byte4_dec'] = df['Byte4'] 

In [79]:
#注意：：処理に時間がかかるので暫くお待ちください！！！

#全てのPandas seriesの型を１０進数に変更する。
#for文と関数がイケてないので処理が遅い。　Series毎キャストするか、変換の他の方法が
#見つかれば改良する。正直この遅さならVBAのExcel関数呼び出しの方が早いとおもわれ。。
#for BPos in df.loc[:,'Byte1':'Byte8']:
for BPos in df.iloc[:,4:]:
    hexToDec(df[BPos] ,df[BPos])

In [26]:
#基本的は1つのデータ解析の中で多数のバイトを変換するケースより、少数（１，２ラベル）を変換する事が多い。
#よってoffset/facを8byte分一括返還するのではなく、必要なbyte番号を指定してその１つだけを変換するようにする。

In [73]:
#変換の方法は20191225現在ではバイト単位でしか出来ない。
#できればfac/offsetの情報はCANDBかエクセルから持って期待が、どうするか。
#ByPos = input("bytepositionを入力してください。") #colmunの名前に結合するのであえてstr型にしている。
#fac = float(input("facを入力してください。"))
#offset =float(input("offsetを入力してください。"))
#unit =input("単位を入力してください。")

#For 1byte message
ByPos = str(7)

#for 2bytes message
ByPosL = str(2)
ByPosH = str(3)

fac = 50
offset = -5000
unit = "rpm"

In [75]:
CAN_TrgLo = "Byte" + ByPosL
CAN_TrgHi = "Byte" + ByPosH
CAN_TrgPosPys2BytePre = "Byte" + ByPosL + "_" + ByPosH + "_pys" + " ["  + unit + "]"

In [74]:
df.head(10)

,Time,Node,CANID,DLC,Byte1,Byte2,Byte3,Byte4,Byte5,Byte6,Byte7,Byte8
0,0.002803,1,CF00400x,8,00,80,7D,00,00,FF,FF,FF
1,0.003303,1,488,8,00,00,00,00,00,00,00,00
2,0.003799,1,512,8,00,00,00,00,00,00,00,00
3,0.004299,1,108,8,00,00,00,00,00,00,00,00
4,0.005067,1,208,8,FF,FE,32,00,00,00,00,32
5,0.005651,1,CF00400x,8,00,80,7D,00,00,FF,FF,FF
6,0.006143,1,348,8,00,32,00,32,00,01,00,00
7,0.006735,1,18FFE000x,8,10,FF,FF,FF,FF,FE,FF,FF
8,0.007751,1,18FFE100x,8,FF,FF,FF,FF,FF,FF,FE,7A
9,0.008251,1,488,8,00,00,00,00,00,00,00,00


In [78]:
#Hexも文字列を結合 LoHiの順になる。 e.g)Lo = FE, Hi = 12 -> result FE12
df[CAN_TrgPosPys2BytePre] = df[CAN_TrgLo] + df[CAN_TrgHi]
df.head()

,Time,Node,CANID,DLC,Byte1,Byte2,Byte3,Byte4,Byte5,Byte6,Byte7,Byte8,Byte2_3_pys [rpm]
0,0.002803,1,CF00400x,8,00,80,7D,00,00,FF,FF,FF,807D
1,0.003303,1,488,8,00,00,00,00,00,00,00,00,0000
2,0.003799,1,512,8,00,00,00,00,00,00,00,00,0000
3,0.004299,1,108,8,00,00,00,00,00,00,00,00,0000
4,0.005067,1,208,8,FF,FE,32,00,00,00,00,32,FE32


AttributeError: 'function' object has no attribute 'head'

In [77]:
#単位変換公式 ：　物理値  = CAN値 * fac  + offset
def conv(CANraw):
    return CANraw * fac + offset
#def CANtoPys(TrgPos):
#    return df[CAN_TrgPos].apply(conv)

In [134]:
#functools.partial(conv, fac=fac,offset = offset ) 

In [135]:
CAN_TrgPos = "Byte"  + ByPos
CAN_TrgPosPys = CAN_TrgPos + "_pys" + " ["  + unit + "]"
df[CAN_TrgPosPys] = df[CAN_TrgPos].apply(conv)
#df[CAN_TrgPosPys] = CANtoPys(CAN_TrgPos)
df[CAN_TrgPosPys].head()

0    7750
1   -5000
2   -5000
3   -5000
4   -5000
Name: Byte7_pys [rpm], dtype: int64

In [136]:

temp = pd.read_csv("Log_DM1_single_out.csv")
temp.head()

,Time,Node,CANID,DLC,Byte1,Byte2,Byte3,Byte4,Byte5,Byte6,Byte7,Byte8
0,0.002803,1,CF00400x,8,00,80,7D,00,00,FF,FF,FF
1,0.003303,1,488,8,00,00,00,00,00,00,00,00
2,0.003799,1,512,8,00,00,00,00,00,00,00,00
3,0.004299,1,108,8,00,00,00,00,00,00,00,00
4,0.005067,1,208,8,FF,FE,32,00,00,00,00,32


In [137]:
df.head()

,Time,Node,CANID,DLC,Byte1,Byte2,Byte3,Byte4,Byte5,Byte6,Byte7,Byte8,Byte8_pys [Nm],Byte7_pys [rpm]
0,0.002803,1,CF00400x,8,0,128,125,0,0,255,255,255,127.5,7750
1,0.003303,1,488,8,0,0,0,0,0,0,0,0,0.0,-5000
2,0.003799,1,512,8,0,0,0,0,0,0,0,0,0.0,-5000
3,0.004299,1,108,8,0,0,0,0,0,0,0,0,0.0,-5000
4,0.005067,1,208,8,255,254,50,0,0,0,0,50,25.0,-5000


In [16]:
#結果をgraphに出力する。
# %matplotlibだとintarativeに動かせないので不採用
"""
x = df.iloc[:,0]
y = df.iloc[:,12]
z = df.iloc[:,11]
plt.plot(x, y, 'b')
#plt.plot(x, z)
plt.title("Pysical value of " + CAN_TrgPos)
plt.xlabel('Time[ms]')
plt.ylabel(unit)
plt.legend(["This is "+CAN_TrgPos])

plt.show()
"""

'\nx = df.iloc[:,0]\ny = df.iloc[:,12]\nz = df.iloc[:,11]\nplt.plot(x, y, \'b\')\n#plt.plot(x, z)\nplt.title("Pysical value of " + CAN_TrgPos)\nplt.xlabel(\'Time[ms]\')\nplt.ylabel(unit)\nplt.legend(["This is "+CAN_TrgPos])\n\nplt.show()\n'

In [17]:
"""
#interactiveに使えるdata visualizationはbokehとplotlyが有名。
#plotlyは優良だが有料なものが多いので今回はbokehで描画
"""


'\n#interactiveに使えるdata visualizationはbokehとplotlyが有名。\n#plotlyは優良だが有料なものが多いので今回はbokehで描画\n'

In [152]:
columnsName = df.columns 
x = df.iloc[:20,0]
y = df.loc[:20,columnsName[12]]
#y = df.iloc[:20,12]
z = df.loc[:20,columnsName[13]]
#z = df.iloc[:20,13]

In [162]:
"""
BokehでのFigureの設定
"""
p = figure(
    title="CAN pysical value", 
    x_axis_label='sec', 
    y_axis_label =unit,
     #y_axis_type="log", 
    #y_range=[0.001, 10**11]
)
#()の外に記載する場合はp.とすることで可能。　
#注釈の場所
p.legend.location = "top_left"
#透明度
#p.grid.grid_line_alpha = 0　

In [163]:
#1つめのデータの設定
#Data analysisにはcircleの方がいい、process pointが分かりやすい。
p.line(x,y,
       legend=columnsName[12],
       line_width = 0.5,
       line_color = "red"
      )

#Line以外で描画したいときのExample
#p.circle(x, x, legend="y=x", fill_color="white", size=8)
#p.line(x, y0, legend="y=x^2", line_width=3)
#p.line(x, y1, legend="y=10^x", line_color="red")
#p.circle(x, y1, legend="y=10^x", fill_color="red", line_color="red", size=6)
#p.line(x, y2, legend="y=10^x^2", line_color="orange", line_dash="4 4")


C:\Program Files\Anaconda3\lib\site-packages\bokeh\models\sources.py:114: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('x', 20), ('y', 21)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))


GlyphRenderer(id='bde1d0ff-3644-4aee-8f61-d993b20ba503', ...)

In [176]:
"""
#2つ目以降のデータを同時に描画したいときはこちらを参照されたし
https://docs.bokeh.org/en/1.0.0/docs/user_guide/plotting.html#userguide-plotting
"""

'\n#2つ目以降のデータを同時に描画したいときはこちらを参照されたし\nhttps://docs.bokeh.org/en/1.0.0/docs/user_guide/plotting.html#userguide-plotting\n'

In [170]:
show(p)

temp

In [129]:
p = figure(
    title="CAN pysical value", 
    x_axis_label='sec', 
    y_axis_label =unit,
     #y_axis_type="log", 
    #y_range=[0.001, 10**11]
    
)


p.line(x,y,
       legend=columnsName[12],
       line_width = 0.5,
       line_color = "red"
      )

p.circle(x,z,
       legend=columnsName[13],
       line_width = 0.5,
         fill_color="white",
         size = 4
         
      )


C:\Program Files\Anaconda3\lib\site-packages\bokeh\models\sources.py:114: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('x', 20), ('y', 21)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))


GlyphRenderer(id='8d85e546-1c5f-4514-b926-a9a9a19e7992', ...)

In [184]:
show(p)

RuntimeError: bokeh sample data directory does not exist, please execute bokeh.sampledata.download()

NameError: name 'bokeh' is not defined